In [1]:
import os
import math
import pandas as pd
import numpy as np
import urllib
import matplotlib.pyplot as plt
%matplotlib inline

import hydrofunctions as hf

import fbprophet 

# For inputting City information to get distance to sensors.
from  geopy.geocoders import Nominatim

# Date time conversion registration.
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()


In [38]:
## Reading in site data from 'Gage_sites.txt'
file = 'Gage_sites.txt'
sites = pd.read_csv(file, sep = '\t', header = 39, skiprows = [40], dtype = {"site_no" : "str"})

site_no = list(sites["site_no"])
sites.head(10)

,site_no,station_nm,lat_va,long_va,coord_acy_cd,coord_datum_cd,dec_lat_va,dec_long_va,coord_acy_cd.1,dec_coord_datum_cd,state_cd,map_nm,alt_va,alt_acy_va,alt_datum_cd,huc_cd,basin_cd,peak_begin_date,peak_end_date,peak_count_nu
0,06614800,"MICHIGAN RIVER NEAR CAMERON PASS, CO",402946.00,1055152.00,F,NAD27,40.496094,-105.865012,F,NAD83,8,"FALL RIVER PASS, CO",10390.00,NaN,NaN,10180001,co01,1974-06-18,2019-07-01,46
1,06620000,"NORTH PLATTE RIVER NEAR NORTHGATE, CO",405611.90,1062021.10,S,NAD83,40.936639,-106.339194,S,NAD83,8,NORTHGATE,7810.39,NaN,NaN,10180001,wy09,1904-05-26,2017-06-13,104
2,06696980,"TARRYALL CREEK AT UPPER STATION NEAR COMO, CO",392022.00,1055440.00,F,NAD27,39.339433,-105.911681,F,NAD83,8,"COMO, CO",9935.00,NaN,NaN,10190001,co01,1978-06-15,2019-06-13,27
3,06700000,"SOUTH PLATTE RIVER ABOVE CHEESMAN LAKE, CO.",390946.00,1051835.00,F,NAD27,39.162769,-105.310273,F,NAD83,8,CHEESMAN LAKE,6845.86,NaN,NaN,10190002,co01,1925-07-23,2019-06-25,36
4,06701620,"TROUT CREEK BLW FERN CR NR WESTCREEK, CO.",391003.00,1050718.00,F,NAD27,39.167491,-105.122211,F,NAD83,8,DAKAN MOUNTAIN,7440.00,NaN,NaN,10190002,co01,2003-05-20,2019-07-26,17
5,06701700,WEST CR. ABV SHREWSBURY GULCH NR WESTCREEK CO,390843.53,1050936.04,1,NAD83,39.145425,-105.160011,1,NAD83,8,"WESTCREEK, CO",7512.00,NaN,NaN,10190002,co01,2003-08-31,2019-04-10,17
6,06701900,SOUTH PLATTE RIVER BLW BRUSH CRK NEAR TRUMBULL...,391536.00,1051317.00,F,NAD27,39.259990,-105.221938,F,NAD83,8,DECKERS CO,6380.00,NaN,NaN,10190002,co01,2002-07-21,2018-07-05,17
7,06708600,"WEST PLUM CREEK NEAR PERRY PARK, CO",391534.10,1045706.85,S,NAD83,39.259472,-104.951903,S,NAD83,8,"DAWSON BUTTE, CO",6340.00,NaN,NaN,10190002,co01,2010-04-29,2019-04-29,10
8,06708690,"WEST PLUM CREEK AT SEDALIA, CO",392545.40,1045804.00,1,NAD83,39.429278,-104.967778,1,NAD83,8,"SEDALIA, CO",5763.00,NaN,NaN,10190002,co01,2015-06-12,2019-04-26,5
9,06708800,"EAST PLUM CR ABV HASKINS GULCH NR CASTLE ROCK, CO",392522.80,1045416.60,5,NAD83,39.423000,-104.904611,5,NAD83,8,"SEDALIA, CO",5960.00,NaN,NaN,10190002,co01,2000-08-31,2018-07-17,19


In [42]:
#site = '09081000'
start = '2000-01-23'
end = '2020-01-23'

for site in site_no[0:2] :
    herring = hf.NWIS(site, 'dv', start, end)
    herring.get_data()
    my_dict = herring.json()
    df = hf.extract_nwis_df(my_dict)
    print(df.head())

            USGS:06614800:00060:00003 USGS:06614800:00060:00003_qualifiers
datetime                                                                  
2000-01-23                       0.34                                  A e
2000-01-24                       0.34                                  A e
2000-01-25                       0.34                                  A e
2000-01-26                       0.34                                  A e
2000-01-27                       0.34                                  A e
            USGS:06620000:00060:00003 USGS:06620000:00060:00003_qualifiers
datetime                                                                  
2000-01-23                       80.0                                  A e
2000-01-24                       80.0                                  A e
2000-01-25                       82.0                                  A e
2000-01-26                       86.0                                  A e
2000-01-27               

In [43]:
help(herring)

Help on NWIS in module hydrofunctions.station object:

class NWIS(Station)
 |  A class for working with data from the USGS NWIS service.
 |  
 |  description
 |  
 |  Args:
 |      site (str or list of strings):
 |          a valid site is '01585200' or ['01585200', '01646502']. Site
 |          should be None if stateCd or countyCd are not None.
 |  
 |      service (str):
 |          can either be 'iv' or 'dv' for instantaneous or daily data.
 |          'dv'(default): daily values. Mean value for an entire day.
 |          'iv': instantaneous value measured at this time. Also known
 |          as 'Real-time data'. Can be measured as often as every
 |          five minutes by the USGS. 15 minutes is more typical.
 |  
 |      start_date (str):
 |         should take on the form yyyy-mm-dd
 |  
 |      end_date (str):
 |          should take on the form yyyy-mm-dd
 |  
 |      stateCd (str):
 |          a valid two-letter state postal abbreviation. Default is None.
 |  
 |      county

In [44]:
herring.start_date

'2000-01-23'

In [6]:
df.head()

,USGS:09081000:00060:00003,USGS:09081000:00060:00003_qualifiers
datetime,,
2000-01-23,252.0,A
2000-01-24,252.0,A
2000-01-25,264.0,A
2000-01-26,271.0,A
2000-01-27,262.0,A


In [16]:
df.plot()
plt.xlabel('Date')
plt.yscale('log')
plt.ylabel('Discharge (cfs)')
plt.title('Daily Mean Discharge for Roaring Fork River')
plt.savefig('RRFork1.png')
plt.close()

In [8]:
df.rename(index=str, columns = {"USGS:09081000:00060:00003" : "y", 
                                "USGS:09081000:00060:00003_qualifiers" : "flags"}, 
         inplace = True)

df['ds'] = df.index[:]

df.head()

,y,flags,ds
datetime,,,
2000-01-23 00:00:00,252.0,A,2000-01-23 00:00:00
2000-01-24 00:00:00,252.0,A,2000-01-24 00:00:00
2000-01-25 00:00:00,264.0,A,2000-01-25 00:00:00
2000-01-26 00:00:00,271.0,A,2000-01-26 00:00:00
2000-01-27 00:00:00,262.0,A,2000-01-27 00:00:00


In [9]:
df_prophet = fbprophet.Prophet(changepoint_prior_scale=0.05, daily_seasonality=True, interval_width = 0.25)
df_prophet.fit(df)

In [10]:
df_forecast = df_prophet.make_future_dataframe(periods=365 * 1, freq='D')
df_forecast = df_prophet.predict(df_forecast)

In [11]:
df_prophet.plot(df_forecast, xlabel = 'Date', ylabel = 'Discharge (cfs)')
plt.title('Discharge at ROARING FORK RIVER NEAR EMMA, CO')
plt.ylim(55,6000)
plt.yscale('log')
plt.savefig('RRFork1_Pred.png')
plt.close()

In [12]:
df_prophet.plot_components(df_forecast)
plt.savefig('RRFork1_Components.png')
plt.close()

In [15]:
help(herring)

Help on NWIS in module hydrofunctions.station object:

class NWIS(Station)
 |  A class for working with data from the USGS NWIS service.
 |  
 |  description
 |  
 |  Args:
 |      site (str or list of strings):
 |          a valid site is '01585200' or ['01585200', '01646502']. Site
 |          should be None if stateCd or countyCd are not None.
 |  
 |      service (str):
 |          can either be 'iv' or 'dv' for instantaneous or daily data.
 |          'dv'(default): daily values. Mean value for an entire day.
 |          'iv': instantaneous value measured at this time. Also known
 |          as 'Real-time data'. Can be measured as often as every
 |          five minutes by the USGS. 15 minutes is more typical.
 |  
 |      start_date (str):
 |         should take on the form yyyy-mm-dd
 |  
 |      end_date (str):
 |          should take on the form yyyy-mm-dd
 |  
 |      stateCd (str):
 |          a valid two-letter state postal abbreviation. Default is None.
 |  
 |      county